In [ ]:
%pip install llama-index llama-index-llms-mistralai llama-index-embeddings-fastembed -q
%pip install llama-index-retrievers-bm25 llama-index-tools-duckduckgo -q

In [ ]:
from llama_index.core.agent.workflow import AgentWorkflow
from llama_index.core import Settings
from llama_index.llms.mistralai import MistralAI 
from llama_index.embeddings.fastembed import FastEmbedEmbedding
import os

os.environ["MISTRAL_API_KEY"] = "API KEY"

Settings.llm = MistralAI(model="mistral-small-latest")
Settings.embed_model = FastEmbedEmbedding(model_name="BAAI/bge-small-en")
Settings.chunk_size = 512
Settings.chunk_overlap = 64

In [ ]:
import datasets
from llama_index.core.schema import Document

In [ ]:
guest_dataset = datasets.load_dataset("agents-course/unit3-invitees", split="train")

In [ ]:
docs = [
    Document(
        text="\n".join([
            f"Name: {guest_dataset['name'][i]}",
            f"Relation: {guest_dataset['relation'][i]}",
            f"Description: {guest_dataset['description'][i]}",
            f"Email: {guest_dataset['email'][i]}"
        ]),
        metadata={"name": guest_dataset['name'][i]}
    )
    for i in range(len(guest_dataset))
]

In [ ]:
from llama_index.tools.duckduckgo import DuckDuckGoSearchToolSpec
from llama_index.core.tools import FunctionTool

tool_spec = DuckDuckGoSearchToolSpec()

search_tool = FunctionTool.from_defaults(tool_spec.duckduckgo_full_search)

In [ ]:
import random
from llama_index.core.tools import FunctionTool
from huggingface_hub import list_models, list_datasets

def get_hub_stats(author: str) -> str:
    """Fetches the most downloaded model from a specific author on the Hugging Face Hub."""
    try:
        
        models = list(list_models(author=author, sort="downloads", direction=-1, limit=1))

        if models:
            model = models[0]
            return f"The most downloaded model by {author} is {model.id} with {model.downloads:,} downloads."
        else:
            return f"No model found for author {author}."
    except Exception as e:
        return f"Error fetching dataset for {author}: {str(e)}"

 
hub_stats_tool = FunctionTool.from_defaults(get_hub_stats)

In [ ]:
import random
from llama_index.core.tools import FunctionTool

def get_weather_info(location: str) -> str:
    """Fetches dummy weather information for a given location."""
    weather_conditions = [
        {"condition": "Rainy", "temp_c": 15},
        {"condition": "Clear", "temp_c": 25},
        {"condition": "Windy", "temp_c": 20}
    ]
    data = random.choice(weather_conditions)
    return f"Weather in {location}: {data['condition']}, {data['temp_c']}°C"

weather_info_tool = FunctionTool.from_defaults(get_weather_info)

In [ ]:
from llama_index.core.tools import FunctionTool
from llama_index.retrievers.bm25 import BM25Retriever

bm25_retriever = BM25Retriever.from_defaults(nodes=docs)

def get_guest_info_retriever(query: str) -> str:
    """Retrieves detailed information about gala guests based on their name or relation."""
    results = bm25_retriever.retrieve(query)
    if results:
        return "\n\n".join([doc.text for doc in results[:3]])
    else:
        return "No matching guest information found."

guest_info_tool = FunctionTool.from_defaults(get_guest_info_retriever)

In [ ]:
alfred = AgentWorkflow.from_tools_or_functions(
    [guest_info_tool, search_tool, weather_info_tool, hub_stats_tool],
)

In [ ]:
query = "Tell me about Lady Ada Lovelace. What's her background?"
response = await alfred.run(query)

print("Response:")
print(response.response.blocks[0].text)

In [ ]:
query = "What's the weather like in Paris tonight? Will it be suitable for our fireworks display?"
response = await alfred.run(query)

print("Response:")
print(response)

In [ ]:
query = "One of our guests is from Google. What can you tell me about their most popular model?"
response = await alfred.run(query)

print("Alfred's Response:")
print(response)

In [ ]:
query = "I need to speak with Dr. Nikola Tesla about recent advancements in wireless energy. Can you help me prepare for this conversation?"
response = await alfred.run(query)

print("Alfred's Response:")
print(response)

In [ ]:
from llama_index.core.workflow import Context

ctx = Context(alfred)


response1 = await alfred.run("Tell me about Lady Ada Lovelace.", ctx=ctx)
print("First Response:")
print(response1)


response2 = await alfred.run("What projects is she currently working on?", ctx=ctx)
print("Second Response:")
print(response2)